| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |

# Working with Mineral Composition Data

### Mineral Database
pyrolite includes a limited mineral database which is useful for looking up endmember compositions.

In [ ]:
import pandas as pd
from pyrolite.mineral.mindb import (
    list_groups,
    list_minerals,
    list_formulae,
    get_mineral,
    get_mineral_group,
)

pd.set_option("precision", 3)  # smaller outputs

From the database, you can get the list of its contents using a few utility
functions:



In [ ]:
list_groups()

In [ ]:
list_minerals()

In [ ]:
list_formulae()

You can also directly get the composition of specific minerals by name:




In [ ]:
get_mineral("forsterite")

If you want to get compositions for all minerals within a specific group, you can
use `get_mineral_group`:



In [ ]:
get_mineral_group("olivine")

### Mineral Endmember Decomposition

A common task when working with mineral chemistry data is to take measured compositions
and decompose these into relative proportions of mineral endmember compositions.
pyrolite includes some utilities to achieve this and a limited mineral database
for looking up endmember compositions.

In [ ]:
import pandas as pd
import numpy as np
from pyrolite.mineral.mindb import get_mineral
from pyrolite.mineral.normative import endmember_decompose

First we'll start with a composition of an unknown olivine:




In [ ]:
comp = pd.Series({"MgO": 42.06, "SiO2": 39.19, "FeO": 18.75})

We can break this down into olivine endmebmers using the
`endmember_decompose` function:




In [ ]:
ed = endmember_decompose(
    pd.DataFrame(comp).T, endmembers="olivine", ord=1, molecular=True
)
ed

Equally, if you knew the likely endmembers beforehand, you could specify a list of
endmembers:




In [ ]:
ed = endmember_decompose(
    pd.DataFrame(comp).T, endmembers=["forsterite", "fayalite"], ord=1, molecular=True
)
ed

We can check this by recombining the components with these proportions. We can first
lookup the compositions for our endmembers:




In [ ]:
em = pd.DataFrame([get_mineral("forsterite"), get_mineral("fayalite")])
em.loc[:, ~(em == 0).all(axis=0)]  # columns not full of zeros

First we have to convert these element-based compositions to oxide-based compositions:




In [ ]:
emvalues = (
    em.loc[:, ["Mg", "Si", "Fe"]]
    .pyrochem.to_molecular()
    .fillna(0)
    .pyrochem.convert_chemistry(to=["MgO", "SiO2", "FeO"], molecular=True)
    .fillna(0)
    .pyrocomp.renormalise(scale=1)
)
emvalues

These can now be used with our endmember proportions to regenerate a composition:




In [ ]:
recombined = pd.DataFrame(ed.values.flatten() @ emvalues).T.pyrochem.to_weight()
recombined

To make sure these compositions are within 0.01 percent:




In [ ]:
assert np.allclose(recombined.values, comp.values, rtol=10 ** -4)

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |